In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Classifiers
#import decisiontreeclassifier
from sklearn import tree
from sklearn.tree import export_text
from sklearn.tree import DecisionTreeClassifier
#import logisticregression classifier
from sklearn.linear_model import LogisticRegression

#import knn classifier
from sklearn.neighbors import KNeighborsClassifier

#for validating your classification model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

# feature selection
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from sklearn.ensemble import RandomForestClassifier

# grid search
from sklearn.model_selection import GridSearchCV

# advanced algorthms
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

#ignore warning
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# load data
cl = pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv')
cl.head()

In [ ]:
# Looking for data type
cl.info()

In [ ]:
# looking for null values in the dataset
cl.isnull().sum()

# Decriptive Analysis and Data Visualization

In [ ]:
# to check outliers
cl.describe()

In [ ]:
# Pivot talbe for output thus those less chance of heart attack and more chance of heart attack.
cl.groupby(['output']).size()

In [ ]:
# plot for output
cl.groupby(['output']).size().plot(kind='bar')

In [ ]:
cl.groupby(['output', 'sex']).size()

In [ ]:
cl.groupby(['output', 'fbs']).size()

In [ ]:
cl.groupby(['output', 'slp']).size()

In [ ]:
# data visualisation for output and age
sns.violinplot(x="sex", y="output", data=cl)

In [ ]:
# data visualisation for output and chol
sns.catplot("output", "chol", data=cl, kind='bar')

# Exploratory Analysis

In [ ]:
# Correlation Analysis without Dummies
cl.corr()

# Correlation Analysis with dummies

In [ ]:
cl.nunique()

In [ ]:
cl_1 = pd.get_dummies(cl, columns=['output', 'cp', 'restecg', 'sex'])
cl_1.head()

In [ ]:
cl_1=cl_1.corr()
cl_1

In [ ]:
# Heat Map
fig = plt.figure(figsize = (30,15))
sns.heatmap(cl_1, annot = True)

## First of all we need to explain the positives and negatives associated with the numbers obtained.
- Positive Correlation; A positive correlation is a relationship between 2 variables which the increase of one variable causes an increase for another variable.
- Negative Correlation; The Negative correlation is the opposite, it’s a relationship between 2 variables which the increase of one variable causes a decrease for another variable. This applies otherwise.
- From our correlation analysis, we can see age has AGE has a positive weak correlation with output_0 (less chance of heart attack). This means that as age increase the more likely there is a chance of heart attack and vice versa.
- We can also see that AGE has a negative weak correlation with output_1 (more chance of heart attack). This means that the lower the age the patient will not have more chance of a heat attack and vice versa.

# Statistical inference


In [ ]:
import scipy.stats as stats
# T-test for clients who is less likely or more likely in relation to age if their mean values are the same
no = cl[cl['output'] == 0]['age']
yes = cl[cl['output'] == 1]['age']

stats.ttest_ind(no, yes)

- Null hypothesis says two means are almost same.
- p-value is a probability if the null hypothesis is true. A high p-value (> 0.05) means we can't reject the null hypothesis
- Since we have a low p-value, we reject the null hypothesis that output and age are the same in terms of painthreshold (No difference)

In [ ]:
# T-test for clients who is less likely or more likely in relation to cp if their mean values are the same
no = cl[cl['output'] == 0]['cp']
yes = cl[cl['output'] == 1]['cp']

stats.ttest_ind(no, yes)

- Null hypothesis says two means are almost same.
- p-value is a probability if the null hypothesis is true. A high p-value (> 0.05) means we can't reject the null hypothesis
- Since we have a low p-value, we reject the null hypothesis that output and cp are the same in terms of painthreshold (No difference)

In [ ]:
# T-test for clients who is less likely or more likely in relation to thalachh if their mean values are the same
no = cl[cl['output'] == 0]['thalachh']
yes = cl[cl['output'] == 1]['thalachh']

stats.ttest_ind(no, yes)

- Null hypothesis says two means are almost same.
- p-value is a probability if the null hypothesis is true. A high p-value (> 0.05) means we can't reject the null hypothesis
- Since we have a low p-value, we reject the null hypothesis that output and thalachh are the same in terms of painthreshold (No difference)

In [ ]:
# T-test for clients who is less likely or more likely in relation to exng if their mean values are the same
no = cl[cl['output'] == 0]['exng']
yes = cl[cl['output'] == 1]['exng']

stats.ttest_ind(no, yes)

- Null hypothesis says two means are almost same.
- p-value is a probability if the null hypothesis is true. A high p-value (> 0.05) means we can't reject the null hypothesis
- Since we have a low p-value, we reject the null hypothesis that output and exng are the same in terms of painthreshold (No difference)

# Model Building

In [ ]:
# looking for unique variables
cl.nunique()

In [ ]:
## Categorical to Dummy Variables
cl =  pd.get_dummies(cl, columns=["sex", "cp", "fbs", "restecg", "exng", "slp", "caa", "thall"],
                         prefix=["sex", "cp", "fbs", "restecg", "exng", "slp", "caa", "thall"],
                         drop_first=True)
cl.head(2)

In [ ]:
### c. checking for p-value (TESTING FOR STATISTICAL SIGNIFICANCE OF INDEPENDENT VARIABLES)
import scipy.stats as stats
cl_corr = pd.DataFrame() # Correlation matrix
cl_p = pd.DataFrame() # Matrix of p-values
for x in cl.columns:   # assuming cl as your dataframe name
   for y in cl.columns:
      corr = stats.pearsonr(cl[x], cl[y])
      cl_corr.loc[x,y] = corr[0]
      cl_p.loc[x,y] = corr[1]

cl_p['output']

### At 0.05 level of significance to test the statistical significance of the X variables, chol, caa_4, thall_1, cp_3 and others are not statistically significant whiles the rest of X variables are statistically significant using the p-value. This is to test our three hypothesis.
### Our null hypthesis is X variables (independent) are not statistically significant in predicting output (dependent variable). We will fail to reject the null hypothesis if the p-value for our x variables are greater than significant level 0.05.
### The p-values of some X variables such as age, thalachh, thall_3 and others are all less than the 0.05 significant level therefore we reject the null hypothesis and that they are all statistically significant in predicting the dependent variable output.

# Using variables that are statistically significant for model selection

In [ ]:
# Assigning dataset into dependent (y_1) and independent (X_1)
y_1 = cl['output']
X_1 = cl.drop(['output', 'chol', 'cp_3', 'fbs_1', 'restecg_2', 'caa_4', 'thall_1'], axis=1)

In [ ]:
# Normalizing all features
from sklearn import preprocessing
X_1 = preprocessing.StandardScaler().fit(X_1).transform(X_1)
X_1[0:5]

In [ ]:
# Splitting data into 70/30. 70% trained dataset and 30% test dataset
X_1_train, X_1_test, y_1_train, y_1_test = train_test_split(X_1, y_1, train_size = 0.7, random_state =100)

In [ ]:
## Best model selection
plt.figure()

# Add the models to the list that you want to view on the ROC plot
models = [
{
    'label': 'Decision Tree',
    'model': DecisionTreeClassifier(),
},
{
    'label': 'K-nearest neighbors',
    'model': KNeighborsClassifier(),
},
{
    'label': 'Logistic Regression',
    'model': LogisticRegression(solver='lbfgs', max_iter=20000),
},
{
    'label': 'Random Forest',
    'model': RandomForestClassifier(n_estimators=100),
}
]

# Below for loop iterates through your models list
for m in models:
    model = m['model'] # select the model
    model.fit(X_1_train, y_1_train) # train the model
    #y_pred=model.predict(X_1_test) # predict the test data
    
    # Compute False postive rate, and True positive rate
    fpr, tpr, thresholds = metrics.roc_curve(y_1_test, model.predict_proba(X_1_test)[:,1])

    # Calculate Area under the curve to display on the plot
    auc = metrics.roc_auc_score(y_1_test,model.predict(X_1_test))

    # Now, plot the computed values
    plt.plot(fpr, tpr, label='%s ROC (area = %0.2f)' % (m['label'], auc))

# Custom settings for the plot 
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-Specificity(False Positive Rate)')
plt.ylabel('Sensitivity(True Positive Rate)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()   # Display

# From our analysis, logistic regression came out on top with 86% using ROC.

In [ ]:
# Assigning dataset into dependent (y) and independent (x)
y = cl['output']
X = cl.drop(['output'], axis=1)

In [ ]:
# Normalizing all features
from sklearn import preprocessing
X_2 = preprocessing.StandardScaler().fit(X).transform(X)
X_2[0:5]

In [ ]:
# Splitting data into 70/30. 70% trained dataset and 30% test dataset
X_train, X_test, y_train, y_test = train_test_split(X_2, y, train_size = 0.7, random_state =100)

In [ ]:
# Model Selection
plt.figure()

# Add the models to the list that you want to view on the ROC plot
models = [
{
    'label': 'Decision Tree',
    'model': DecisionTreeClassifier(),
},
{
    'label': 'K-nearest neighbors',
    'model': KNeighborsClassifier(),
},
{
    'label': 'Logistic Regression',
    'model': LogisticRegression(solver='lbfgs', max_iter=20000),
},
{
    'label': 'Random Forest',
    'model': RandomForestClassifier(n_estimators=100),
}
]

# Below for loop iterates through your models list
for m in models:
    model = m['model'] # select the model
    model.fit(X_train, y_train) # train the model
    #y_pred=model.predict(X_test) # predict the test data
    
    # Compute False postive rate, and True positive rate
    fpr, tpr, thresholds = metrics.roc_curve(y_test, model.predict_proba(X_test)[:,1])

    # Calculate Area under the curve to display on the plot
    auc = metrics.roc_auc_score(y_test,model.predict(X_test))

    # Now, plot the computed values
    plt.plot(fpr, tpr, label='%s ROC (area = %0.2f)' % (m['label'], auc))

# Custom settings for the plot 
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-Specificity(False Positive Rate)')
plt.ylabel('Sensitivity(True Positive Rate)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()   # Display

# From our analysis, logistic regression came out on top with 87% using ROC.

In [ ]:
# Model Selection for advance classification algorithms
plt.figure()

# Add the models to the list that you want to view on the ROC plot
models = [
{
    'label': 'Support Vector Machine',
    'model': SVC(gamma='auto', probability=True),
},
{
    'label': 'Gradient Boosting',
    'model': GradientBoostingClassifier(),
},
{
    'label': 'Neural Network',
    'model': MLPClassifier(solver='lbfgs', max_iter=15000),
}
]

# Below for loop iterates through your models list
for m in models:
    model = m['model'] # select the model
    model.fit(X_train, y_train) # train the model
    #y_pred=model.predict(X_test) # predict the test data
    
    # Compute False postive rate, and True positive rate
    fpr, tpr, thresholds = metrics.roc_curve(y_test, model.predict_proba(X_test)[:,1])

    # Calculate Area under the curve to display on the plot
    auc = metrics.roc_auc_score(y_test,model.predict(X_test))

    # Now, plot the computed values
    plt.plot(fpr, tpr, label='%s ROC (area = %0.2f)' % (m['label'], auc))

# Custom settings for the plot 
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-Specificity(False Positive Rate)')
plt.ylabel('Sensitivity(True Positive Rate)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()   # Display

# From our analysis, Support Vector Machine came out on top with 85% using ROC.

In [ ]:
# using Logistic Regression
lr = LogisticRegression(solver='lbfgs', max_iter=20000)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
# print coefficients
coef = pd.DataFrame(list(zip(X.columns, np.transpose(lr.coef_))), 
             columns=['X variables', 'coef']).sort_values('coef', ascending=False)

coef['coef'] = coef['coef'].str[0]
coef

In [ ]:
### As we can see, the highest positive coefficient or weight is cp_2 and the highest negative coefficient is caa_1.

In [ ]:
print(metrics.accuracy_score(y_test, lr.predict(X_test)))
print(metrics.confusion_matrix(y_test, lr.predict(X_test)))
print(metrics.classification_report(y_test, lr.predict(X_test)))
print(metrics.roc_auc_score(y_test, lr.predict(X_test)))

In [ ]:
# 10 fold cross validation evaluation
scores = cross_val_score(LogisticRegression(solver='lbfgs', max_iter=20000), X_test, y_test, scoring='accuracy', cv=10)
print(scores)
print(scores.mean())

In [ ]:
# probability of heart attack
probs = lr.predict_proba(X_test)
probs[0:20]
#1st column: probability of less chance of heart attack
#2nd column: probabiility of more chance of heart attack

In [ ]:
# RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100)    #building 100 decision trees
clf=clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
# Evaluation metrics
print(metrics.accuracy_score(y_test, clf.predict(X_test))) #overall accuracy
print(metrics.confusion_matrix(y_test, clf.predict(X_test)))
print(metrics.classification_report(y_test, clf.predict(X_test)))

In [ ]:
# 10 fold cross validation evaluation
scores = cross_val_score(RandomForestClassifier(), X_test, y_test, scoring='accuracy', cv=10)
print(scores)
print(scores.mean())

In [ ]:
# another method
pd.DataFrame(clf.feature_importances_, index = X.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)

- The 5 most important features are oldpeak, thalachh, thall_3, chol and trtbps.

In [ ]:
# Visualization for the most important features.
pd.DataFrame(clf.feature_importances_, index = X.columns,
                                    columns=['importance']).sort_values('importance', ascending=True).plot(kind='barh', 
                                                                                                            legend=None);

In [ ]:
#Support Vector Machine (SVM)
svm = SVC(gamma='auto', probability=True)
svm.fit(X_train, y_train)

In [ ]:
#Model evaluation
# http://scikit-learn.org/stable/modules/model_evaluation.html
print(metrics.accuracy_score(y_test, svm.predict(X_test)))
print("--------------------------------------------------------")
print(metrics.confusion_matrix(y_test, svm.predict(X_test))) 
print("--------------------------------------------------------")
print(metrics.classification_report(y_test, svm.predict(X_test)))
print("--------------------------------------------------------")
print(metrics.roc_auc_score(y_test, svm.predict(X_test)))

In [ ]:
# 10-fold cross-validation
svm = SVC(gamma='auto')

scores = cross_val_score(svm, X_test, y_test, scoring='accuracy', cv=10)
print(scores)
print(scores.mean())

# Model Deployment
- We going to employ the logistic regression model since it gave as the highest prediction.
- Also, the whole dataset is going to used for out model deployment.

In [ ]:
# Logistic Regression
y_predict = lr.predict(X_2)
y_predict[0:20]

In [ ]:
# probability of heart attack
probs_new = lr.predict_proba(X_2)
probs_new[0:20]
#1st column: probability of less chance of heart attack
#2nd column: probabiility of more chance of heart attack

In [ ]:
# Evaluation Metrics
print(metrics.accuracy_score(y, lr.predict(X_2)))
print(metrics.confusion_matrix(y, lr.predict(X_2)))
print(metrics.classification_report(y, lr.predict(X_2)))
print(metrics.roc_auc_score(y, lr.predict(X_2)))

In [ ]:
# 10 fold cross validation evaluation
scores = cross_val_score(LogisticRegression(solver='lbfgs', max_iter=20000), X_2, y, scoring='accuracy', cv=10)
print(scores)
print(scores.mean())

# Evaluation of prediction to original output

In [ ]:
from sklearn.metrics import f1_score
f1_score(y, y_predict, average='weighted') 

# The F1 score is the harmonic average of the precision and recall, where an F1 score reaches its best value at 1 (perfect precision and recall) and worst at 0. 
# It is a good way to show that a classifer has a good value for both recall and precision.
# And finally, we can tell the average accuracy for this classifier is the average of the F1-score for both labels, which is 0.89 in our case.

In [ ]:
from sklearn.metrics import jaccard_score
jaccard_score(y, y_predict,pos_label=1) 
# Lets try jaccard index for accuracy evaluation. 
# we can define jaccard as the size of the intersection divided by the size of the union of two label sets. 
# If the entire set of predicted labels for a sample strictly match with the true set of labels, then the subset accuracy is 1.0; otherwise it is 0.0.

In [ ]:
from sklearn.metrics import log_loss
log_loss(y, probs_new) # This probability is a value between 0 and 1. 
                      # Log loss( Logarithmic loss) measures the performance of a classifier where the predicted output is a probability value between 0 and 1.

In [ ]:
# ROC
preds = lr.predict_proba(X_2)[:,1]
fpr, tpr, threshold = metrics.roc_curve(y, y_predict)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1.05])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()